In [1]:
import os
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.documents import Document
from flashrank import Ranker, RerankRequest

# Load environment variables
load_dotenv()
MINIML_MODEL_PATH = os.getenv("MINIML_MODEL_PATH")

# Sample text
sample_text = '''Artificial Intelligence refers to the simulation of human intelligence in machines that are programmed to think like humans and mimic their actions.
        AI is used in fields such as healthcare (diagnostic tools), finance (fraud detection), and customer service (chatbots).
        Key types include Narrow AI (focused on specific tasks like voice assistants) and General AI (hypothetical AI capable of performing any intellectual task a human can do).
        '''

# 1. Create Document objects
doc_object = Document(page_content=sample_text, metadata={"source": "manual_input"})

# 2. Chunk the text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=30)
chunks = text_splitter.split_documents([doc_object])

print(f"Created {len(chunks)} chunks")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: {chunk.page_content[:100]}...")

# 3. Initialize Embeddings
embeddings = HuggingFaceEmbeddings(model_name=MINIML_MODEL_PATH)

# 4. Create persist directory
current_dir = os.getcwd()
persist_directory = os.path.join(current_dir, "resources", "databases", "db1", "chroma_db")
os.makedirs(persist_directory, exist_ok=True)

# 5. Create Chroma Vector DB
print("\nCreating Chroma vector database...")
chroma_db = Chroma.from_documents(
    documents=chunks, 
    embedding=embeddings, 
    persist_directory=persist_directory
)
print("Chroma vector database created and stored locally.")



Created 3 chunks
Chunk 1: Artificial Intelligence refers to the simulation of human intelligence in machines that are programm...
Chunk 2: AI is used in fields such as healthcare (diagnostic tools), finance (fraud detection), and customer ...
Chunk 3: Key types include Narrow AI (focused on specific tasks like voice assistants) and General AI (hypoth...


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]


Creating Chroma vector database...
Chroma vector database created and stored locally.


In [2]:
# 6. Initialize FlashRank (just once)
print("\nInitializing FlashRank reranker...")
ranker = Ranker(model_name="ms-marco-MiniLM-L-12-v2")

# 7. Basic Retrieval (without reranking)
query = "What are the machines where AI is used?"
print(f"\n--- Basic Retrieval (without reranking) ---")
print(f"Query: {query}")
basic_docs = chroma_db.similarity_search(query, k=5)
print(f"Found {len(basic_docs)} documents")
for i, doc in enumerate(basic_docs):
    print(f"Result {i+1}: {doc.page_content[:100]}...")

# 8. Implement Reranking using FlashRank's native pattern
print(f"\n--- Retrieval with FlashRank Reranking (Native Pattern) ---")

def retrieve_and_rerank(query, vector_db, ranker, initial_k=10, final_k=3):
    """
    Retrieve documents and rerank them using FlashRank
    
    Args:
        query: Search query
        vector_db: Vector database
        ranker: Initialized FlashRank ranker
        initial_k: Number of documents to retrieve initially
        final_k: Number of documents to return after reranking
    
    Returns:
        List of reranked documents with scores
    """
    # Step 1: Retrieve initial documents from vector DB
    initial_docs = vector_db.similarity_search(query, k=initial_k)
    
    # Step 2: Convert to FlashRank passage format
    passages = []
    for i, doc in enumerate(initial_docs):
        passages.append({
            "id": i,  # Simple numeric index
            "text": doc.page_content,
            "meta": doc.metadata  # Optional metadata
        })
    
    # Step 3: Create rerank request and get results
    rerank_request = RerankRequest(query=query, passages=passages)
    results = ranker.rerank(rerank_request)
    
    # Step 4: Convert back to Document objects with scores
    reranked_docs = []
    for result in results[:final_k]:
        original_idx = result["id"]
        doc = initial_docs[original_idx]
        
        # Add score to metadata
        doc.metadata["relevance_score"] = result["score"]
        doc.metadata["rank"] = result.get("rank", 0)
        
        reranked_docs.append(doc)
    
    return reranked_docs, results  # Return both docs and raw results




Initializing FlashRank reranker...

--- Basic Retrieval (without reranking) ---
Query: What are the machines where AI is used?
Found 5 documents
Result 1: AI is used in fields such as healthcare (diagnostic tools), finance (fraud detection), and customer ...
Result 2: AI is used in fields such as healthcare (diagnostic tools), finance (fraud detection), and customer ...
Result 3: AI is used in fields such as healthcare (diagnostic tools), finance (fraud detection), and customer ...
Result 4: AI is used in fields such as healthcare (diagnostic tools), finance (fraud detection), and customer ...
Result 5: AI is used in fields such as healthcare (diagnostic tools), finance (fraud detection), and customer ...

--- Retrieval with FlashRank Reranking (Native Pattern) ---


In [3]:
# Test the reranking
reranked_docs, raw_results = retrieve_and_rerank(query, chroma_db, ranker)

print(f"Query: '{query}'")
print(f"\nReranked Results (top {len(reranked_docs)}):")
for i, doc in enumerate(reranked_docs):
    score = doc.metadata.get('relevance_score', 'N/A')
    print(f"\nResult {i+1} (Score: {score:.4f}):")
    print(f"  Text: {doc.page_content[:150]}...")
    print(f"  Metadata: {doc.metadata}")

# Print raw FlashRank results to see the full output format
print(f"\nRaw FlashRank Results:")
for i, result in enumerate(raw_results[:3]):
    print(f"  Result {i+1}: ID={result['id']}, Score={result['score']:.4f}")

# 9. Compare with and without reranking
print(f"\n--- Comparison: Without vs With Reranking ---")

def compare_approaches(query, k=3):
    print(f"Query: '{query}'")
    
    # Without reranking
    without = chroma_db.similarity_search(query, k=k)
    print(f"\n1. WITHOUT RERANKING (Similarity Search):")
    for i, doc in enumerate(without):
        print(f"   {i+1}. {doc.page_content[:100]}...")
    
    # With reranking
    with_rerank, _ = retrieve_and_rerank(query, chroma_db, ranker, initial_k=10, final_k=k)
    print(f"\n2. WITH FLASHRANK RERANKING:")
    for i, doc in enumerate(with_rerank):
        score = doc.metadata.get('relevance_score', 'N/A')
        print(f"   {i+1}. Score: {score:.4f} - {doc.page_content[:100]}...")

compare_approaches(query)

# 10. Test with multiple queries
print(f"\n--- Testing Multiple Queries ---")

test_queries = [
    "What are the applications of AI in healthcare?",
    "Explain Narrow AI and General AI",
    "How do machines mimic human intelligence?",
    "What is AI used for in finance?",
]

for test_query in test_queries:
    print(f"\n{'-'*50}")
    print(f"Query: '{test_query}'")
    
    # Get reranked results
    reranked, raw = retrieve_and_rerank(test_query, chroma_db, ranker, final_k=2)
    
    print("Top 2 reranked results:")
    for i, doc in enumerate(reranked):
        score = doc.metadata.get('relevance_score', 'N/A')
        print(f"{i+1}. (Score: {score:.4f}) {doc.page_content[:120]}...")

# 11. Advanced: Analyze reranking scores
print(f"\n--- Score Analysis ---")

def analyze_reranking(query, vector_db, ranker, k=10):
    """Analyze how reranking changes document order"""
    
    # Get initial documents
    initial_docs = vector_db.similarity_search(query, k=k)
    
    # Get reranked results
    reranked_docs, raw_results = retrieve_and_rerank(query, vector_db, ranker, initial_k=k, final_k=k)
    
    print(f"Analysis for query: '{query}'")
    print(f"{'Original Rank':<15} {'New Rank':<15} {'Score':<10} {'Text Preview'}")
    print("-" * 70)
    
    # Create a mapping from original index to new rank
    new_rank_map = {}
    for new_idx, result in enumerate(raw_results):
        original_idx = result["id"]
        new_rank_map[original_idx] = (new_idx, result["score"])
    
    # Print analysis
    for original_idx, doc in enumerate(initial_docs):
        if original_idx in new_rank_map:
            new_idx, score = new_rank_map[original_idx]
            change = new_idx - original_idx
            change_symbol = "↑" if change < 0 else "↓" if change > 0 else "="
            print(f"{original_idx+1:<15} {new_idx+1:<15} {score:.4f}   {change_symbol} {doc.page_content[:50]}...")

analyze_reranking(query, chroma_db, ranker)



Query: 'What are the machines where AI is used?'

Reranked Results (top 3):

Result 1 (Score: 0.9975):
  Text: AI is used in fields such as healthcare (diagnostic tools), finance (fraud detection), and customer service (chatbots)....
  Metadata: {'source': 'manual_input', 'relevance_score': 0.9974643, 'rank': 0}

Result 2 (Score: 0.9975):
  Text: AI is used in fields such as healthcare (diagnostic tools), finance (fraud detection), and customer service (chatbots)....
  Metadata: {'source': 'manual_input', 'relevance_score': 0.9974643, 'rank': 0}

Result 3 (Score: 0.9975):
  Text: AI is used in fields such as healthcare (diagnostic tools), finance (fraud detection), and customer service (chatbots)....
  Metadata: {'source': 'manual_input', 'relevance_score': 0.9974643, 'rank': 0}

Raw FlashRank Results:
  Result 1: ID=0, Score=0.9975
  Result 2: ID=1, Score=0.9975
  Result 3: ID=2, Score=0.9975

--- Comparison: Without vs With Reranking ---
Query: 'What are the machines where AI is used

In [4]:
# 12. Create a reusable retriever class
print(f"\n--- Creating Reusable FlashRank Retriever ---")

class FlashRankRetriever:
    """A reusable retriever with built-in FlashRank reranking"""
    
    def __init__(self, vector_db, model_name="ms-marco-MiniLM-L-12-v2"):
        self.vector_db = vector_db
        self.ranker = Ranker(model_name=model_name)
    
    def search(self, query, initial_k=10, final_k=3):
        """Search and rerank documents"""
        docs, raw = retrieve_and_rerank(
            query, 
            self.vector_db, 
            self.ranker, 
            initial_k=initial_k, 
            final_k=final_k
        )
        return docs
    
    def search_with_scores(self, query, initial_k=10, final_k=3):
        """Search and return both documents and raw scores"""
        return retrieve_and_rerank(query, self.vector_db, self.ranker, initial_k, final_k)

# Test the reusable retriever
retriever = FlashRankRetriever(chroma_db)

print("Testing reusable retriever:")
results = retriever.search("AI applications", initial_k=8, final_k=2)
for i, doc in enumerate(results):
    score = doc.metadata.get('relevance_score', 'N/A')
    print(f"Result {i+1} (Score: {score:.4f}): {doc.page_content[:100]}...")

# 13. Bonus: Compare different FlashRank models
print(f"\n--- Comparing Different FlashRank Models ---")

def compare_models(query, vector_db):
    """Compare different FlashRank models"""
    models = [
        "ms-marco-MiniLM-L-12-v2",  # Default, best quality
        "ms-marco-MiniLM-L-6-v2",   # Smaller, faster
        "rank-T5-flan",              # Alternative architecture
    ]
    
    print(f"Query: '{query}'")
    
    for model_name in models:
        print(f"\nModel: {model_name}")
        try:
            ranker = Ranker(model_name=model_name)
            reranked, raw = retrieve_and_rerank(query, vector_db, ranker, final_k=2)
            
            for i, doc in enumerate(reranked):
                score = doc.metadata.get('relevance_score', 'N/A')
                print(f"  {i+1}. Score: {score:.4f} - {doc.page_content[:80]}...")
        except Exception as e:
            print(f"  Error with model {model_name}: {e}")

# Compare models (uncomment to run, might download models)
# compare_models(query, chroma_db)

print("\n✅ FlashRank implementation complete!")


--- Creating Reusable FlashRank Retriever ---
Testing reusable retriever:
Result 1 (Score: 0.9990): AI is used in fields such as healthcare (diagnostic tools), finance (fraud detection), and customer ...
Result 2 (Score: 0.9990): AI is used in fields such as healthcare (diagnostic tools), finance (fraud detection), and customer ...

--- Comparing Different FlashRank Models ---

✅ FlashRank implementation complete!
